In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.io


import plotly.express as px
import matplotlib.pyplot as plt

xls = pd.ExcelFile('cc_spectral_signature.xlsx')
print(xls.sheet_names)

df = pd.read_excel(io='cc_spectral_signature.xlsx', sheet_name='Average Spectra')
del df['Unnamed: 0']
display(df.describe())

df = df.apply(func=lambda e: (e - np.min(e)) / (np.max(e) - np.min(e)), axis=0)
display(df.describe())


In [ ]:
fig = px.line(data_frame=df)
fig

In [ ]:
from hsi_datasets import plot_datacube_image
from hsi_datasets import datacube_generator

endmembers = df.values.T


for seq in range(0,10):    
    file_name = f"abundances/abundances_gaussianfields_spheric_5em_16x16_{(seq+1):03}.mat"
    print(file_name)
    matfile = scipy.io.loadmat(file_name)    
    abundances = matfile['abundances']

    fig, axs = plt.subplots(1,5, figsize=(20,10))
    for endmember_num in range(0,5):
        axs[endmember_num].imshow(abundances[...,endmember_num])        
        axs[endmember_num].set_axis_off()    
    plt.tight_layout()
    plt.show()

    # -------
    
    results = datacube_generator(endmembers_names=list(df.columns), endmembers_spectra=endmembers, abundances_tensor=abundances)
    datacube, abundances_tensor, endmembers_spectra, endmembers_names = results

    outputfile = f"datacube_{seq:03}.png"
    fig = plot_datacube_image(
        datacube=datacube, 
        outputfile=outputfile, 
        showfigure=True, 
        reduce_factors=(1,1,1)
    )